In [3]:
import paho.mqtt.client as mqtt
import json
import ast
import re

SERVER = '147.228.124.230'  # RPi
TOPIC = 'ite/#'
#list teplot
blue_temp = [10]
red_temp = [10]
green_temp = [10]
orange_temp = [10]
black_temp = [10]
pink_temp = [10]
yellow_temp = [10]

time_blue = " "
time_black = " "
time_red = " "
time_pink = " "
time_orange = " "
time_yellow = " "

teplota_blue = 0
teplota_black = 0
teplota_pink = 0
teplota_red = 0
teplota_orange = 0
teplota_yellow = 0

prumer_blue = 0
prumer_black = 0
prumer_pink = 0
prumer_red = 0
prumer_orange = 0
prumer_yellow = 0
# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, mid, qos):
    print('Connected with result code qos:', str(qos))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(TOPIC)


# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    
    global time_blue, teplota_blue, temp_blue, prumer_blue, time_red, teplota_red, temp_red, prumer_red,time_black, teplota_black, temp_black, prumer_black
    global time_pink, teplota_pink, temp_pink, prumer_pink, time_orange, teplota_orange, temp_orange, prumer_orange,time_yellow, teplota_yellow, temp_yellow, prumer_yellow
    if (msg.payload == 'Q'):
        client.disconnect()
    print(msg.payload)
    try:
        msg_dec = str(msg.payload.decode("utf-8","ignore"))
        msg_dec = msg_dec.replace("'", '"')
       # msg_dec = str(getCorrectData(msg_dec))
        msg_dict = json.loads(msg_dec)  # vraci slovnik, parametr je json file
        if (team(msg_dict) == "blue"):
            blue_temp.append(teplota(msg_dict))
            time_blue = cas(msg_dict)
            prumer_blue = prumer(blue_temp)
            teplota_blue = teplota(msg_dict)
        
        if (team(msg_dict) == "black"):
            black_temp.append(teplota(msg_dict))
            time_black = cas(msg_dict)
            prumer_black = prumer(black_temp)
            teplota_black = teplota(msg_dict)
        
        if (team(msg_dict) == "pink"):
            pink_temp.append(teplota(msg_dict))
            time_pink = cas(msg_dict)
            prumer_pink = prumer(pink_temp)
            teplota_pink = teplota(msg_dict)
        
        if (team(msg_dict) == "red"):
            red_temp.append(teplota(msg_dict))
            time_red = cas(msg_dict)
            prumer_red = prumer(red_temp)
            teplota_red = teplota(msg_dict)
        
        if (team(msg_dict) == "orange"):
            orange_temp.append(teplota(msg_dict))
            time_orange = cas(msg_dict)
            prumer_orange = prumer(orange_temp)
            teplota_orange = teplota(msg_dict)
        
        if (team(msg_dict) == "yellow"):
            yellow_temp.append(teplota(msg_dict))
            time_yellow = cas(msg_dict)
            prumer_yellow = prumer(yellow_temp)
            teplota_yellow = teplota(msg_dict)
        
        
        json_data = {"blue" : {"prumerna": prumer_blue,
                                      "maximalni": max(blue_temp),
                                      "minimalni": min(blue_temp),
                                      "posledni": blue_temp[len(blue_temp)-1],
                                      "cas": time_blue},
                    "black" : {"prumerna": prumer_black,
                                      "maximalni": max(black_temp), 
                                      "minimalni": min(black_temp),
                                      "posledni": black_temp[len(black_temp)-1],
                                      "cas": time_black},
                    "pink" : {"prumerna": prumer_pink,
                                     "maximalni": max(pink_temp),
                                     "minimalni": min(pink_temp),
                                     "posledni": pink_temp[len(pink_temp)-1],
                                     "cas": time_pink},
                    "red" : {"prumerna": prumer_red,
                                    "maximalni": max(red_temp),
                                    "minimalni": min(red_temp),
                                    "posledni ": red_temp[len(red_temp)-1],
                                    "cas": time_red},
                    "orange" : {"prumerna": prumer_orange,
                                        "maximalni": max(orange_temp),
                                        "minimalni": min(orange_temp),
                                        "posledni": orange_temp[len(orange_temp)-1],
                                        "cas": time_orange},
                    "yellow" : {"prumerna": prumer_yellow,
                                        "maximalni": max(yellow_temp),
                                        "minimalni": min(yellow_temp),
                                        "posledni": yellow_temp[len(yellow_temp)-1],
                                        "cas": time_yellow}}
       # json_data = ast.literal_eval(json_data)
        json_file = json.dumps(json_data)
        with open("./Webovka/assets/data.json", 'w') as fout:    
            fout.write( json_file)   
            fout.close()   
    except ValueError:
        print('Incorrect data')
   
        
def prumer(data):
    delka = len(data)
    soucet = sum(data)
    return (soucet/delka)
    
def team(dict):
    team_name = dict['team_name']
    return team_name

def teplota(dict):
    temp = dict['temperature']
    return temp


def cas(dict):
    cas = dict['created_on'].replace("\'", "\"")
    return cas
'''
def getCorrectData(str):
    list_str = re.sub('{','',str)
    list_str =  re.sub('{','',list_str)
    list_str =  re.sub('}','',list_str)
    list_str =  re.sub(':','',list_str)
    list_str =  re.sub(',','',list_str)
    list_old = list_str.split(" ")
    list_new = ["'source'" , "'fake'" , "'team_name'", list_old[3], "'created_on'", list_old[5], "'temperature'", list_old[7]]
    str_new = "b"+"\'"+"{"+list_new[0]+": "+list_new[1]+", "+list_new[2]+": "+list_new[3]+", "+list_new[4]+": "+list_new[5]+","+list_new[6]+": "+list_new[7]+"}"+"\'"
    return str_new
'''
def main():
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message

    client.username_pw_set('mqtt_student', password='pivo')

    client.connect(SERVER, 1883, 60)

    # Blocking call that processes network traffic, dispatches callbacks and
    # handles reconnecting.
    # Other loop*() functions are available that give a threaded interface and
    # a manual interface.
    client.loop_forever()


if __name__ == '__main__':
    main()

Connected with result code qos: 0
b'{"source": "fake", "team_name": "red", "created_on": "2020-05-04T18:16:38.829729", "temperature": 21.793270172297866}'
b'{"source": "fake", "team_name": "green", "created_on": "2020-05-04T18:16:40.649748", "temperature": 12.44859660175419}'
b'{"source": "fake", "team_name": "black", "created_on": "2020-05-04T18:16:56.543548", "temperature": 21.365027495897433}'
b'{"source": "fake", "team_name": "pink", "created_on": "2020-05-04T18:16:56.909885", "temperature": 12.157811540998836}'
b'{"source": "fake", "team_name": "yellow", "created_on": "2020-05-04T18:17:01.423081", "temperature": 8.501990633451207}'
b"{'source': 'fake', 'team_name': 'blue', lcreated_on': '2020-05-04T18:17:06.344034', 'temperature': 10.031566226396887}"
Incorrect data
b'{"source": "fake", "team_name": "orange", "created_on": "2020-05-04T18:17:10.095520", "temperature": 15.954975740136783}'
b'{"source": "fake", "team_name": "red", "created_on": "2020-05-04T18:17:38.929426", "temperat

KeyboardInterrupt: 